In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import os
import json
from pandas.io.json import json_normalize

pd.options.display.max_columns = None

In [2]:
def load_df(csv_path, nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    types = {
        "fullVisitorId": "str", # readme says it should be str
        "channelGrouping": "str",
        "date": "str",
        "socialEngagementType": "str",
        "visitId": "int32",
        "visitNumber": "int8",
        "visitStartTime": "int32",
    }
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype=types,
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
        
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [3]:
df_raw = load_df('train_v2.csv', nrows=500000)

Loaded train_v2.csv. Shape: (500000, 60)


In [4]:
display(df_raw.head())

,channelGrouping,customDimensions,date,fullVisitorId,hits,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,device.browserVersion,device.deviceCategory,device.flashVersion,device.isMobile,device.language,device.mobileDeviceBranding,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.mobileDeviceModel,device.mobileInputSelector,device.operatingSystem,device.operatingSystemVersion,device.screenColors,device.screenResolution,geoNetwork.city,geoNetwork.cityId,geoNetwork.continent,geoNetwork.country,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.networkLocation,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,3162355547410993243,"[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,1508198450,1,1508198450,Firefox,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Europe,Germany,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,Western Europe,1,1,1,1,1,NaN,NaN,NaN,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,water bottle,organic,NaN,google
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,8934116514970143966,"[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,1508176307,6,1508176307,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Chrome OS,not available in demo dataset,not available in demo dataset,not available in demo dataset,Cupertino,not available in demo dataset,Americas,United States,not available in demo dataset,not available in demo dataset,San Francisco-Oakland-San Jose CA,(not set),not available in demo dataset,California,Northern America,NaN,2,NaN,2,2,28,NaN,NaN,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,NaN,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,7992466427990357681,"[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,1508201613,1,1508201613,Chrome,not available in demo dataset,not available in demo dataset,mobile,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Americas,United States,not available in demo dataset,not available in demo dataset,not available in demo 

In [5]:
# drop columns that only have one value + customDimensions and hits
one_value_col = [c for c in df_raw.columns if df_raw[c].nunique(dropna=False)==1]
df_raw.drop(one_value_col+['customDimensions', 'hits'], axis=1, inplace=True)

In [6]:
df_test = load_df('test_v2.csv')

Loaded test_v2.csv. Shape: (401589, 59)


In [7]:
df_raw.drop(columns=['trafficSource.campaignCode'], inplace=True)
df_test.drop(one_value_col+["customDimensions", "hits"], axis=1, inplace=True)

In [8]:
tar = 'totals.transactionRevenue'
df_raw[tar].fillna(0, inplace=True)
df_raw[tar] = df_raw[tar].astype(float)
df_raw[tar] = np.log1p(df_raw[tar])

In [9]:
# helper functions pull from fastai library to help add datatime features

import re
def ifnone(a,b):
    "`a` if `a` is not None, otherwise `b`."
    return b if a is None else a

def make_date(df, date_field:str):
    "Make sure `df[field_name]` is of the right date type."
    field_dtype = df[date_field].dtype
    if isinstance(field_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        field_dtype = np.datetime64
    if not np.issubdtype(field_dtype, np.datetime64):
        df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)

def add_datepart(df, field_name, prefix=None, drop=True, time:bool=False):
    "Helper function that adds columns relevant to a date in the column `field_name` of `df`."
    make_date(df, field_name)
    field = df[field_name]
    prefix = ifnone(prefix, re.sub('[Dd]ate$', '', field_name))
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start', 
            'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[prefix + n] = getattr(field.dt, n.lower())
    df[prefix + 'Elapsed'] = field.astype(np.int64) // 10 ** 9
    if drop: df.drop(field_name, axis=1, inplace=True)
    return df

In [10]:
df_raw["visitStartTime"] = pd.to_datetime(df_raw["visitStartTime"], infer_datetime_format=True, unit="s")
df_raw["date"] = pd.to_datetime(df_raw["date"], infer_datetime_format=True, format="%Y%m%d")
add_datepart(df_raw, 'date')
add_datepart(df_raw, 'visitStartTime')

,channelGrouping,fullVisitorId,visitId,visitNumber,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,visitStartTimeYear,visitStartTimeMonth,visitStartTimeWeek,visitStartTimeDay,visitStartTimeDayofweek,visitStartTimeDayofyear,visitStartTimeIs_month_end,visitStartTimeIs_month_start,visitStartTimeIs_quarter_end,visitStartTimeIs_quarter_start,visitStartTimeIs_year_end,visitStartTimeIs_year_start,visitStartTimeElapsed
0,Organic Search,3162355547410993243,1508198450,1,Firefox,desktop,False,Windows,not available in demo dataset,Europe,Germany,not available in demo dataset,(not set),not available in demo dataset,Western Europe,1,1,1,1,1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,water bottle,organic,NaN,google,2017,10,42,16,0,289,False,False,False,False,False,False,1508112000,2017,10,42,17,1,290,False,False,False,False,False,False,1508198450
1,Referral,8934116514970143966,1508176307,6,Chrome,desktop,False,Chrome OS,Cupertino,Americas,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,NaN,2,NaN,2,2,28,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,NaN,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com,2017,10,42,16,0,289,False,False,False,False,False,False,1508112000,2017,10,42,16,0,289,False,False,False,False,False,False,1508176307
2,Direct,7992466427990357681,1508201613,1,Chrome,mobile,True,Android,not available in demo dataset,Americas,United States,not available in demo dataset,windjammercable.net,not available in demo dataset,Northern America,NaN,2,1,2,1,38,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not set),True,NaN,(none),NaN,(direct),2017,10,42,16,0,289,False,False,False,False,False,False,1508112000,2017,10,42,17,1,290,False,False,False,False,False,False,1508201613
3,Organic Search,9075655783635761930,1508169851,1,Chrome,desktop,False,Windows,not available in demo dataset,Asia,Turkey,not available in demo dataset,unknown.unknown,not available in demo dataset,Western Asia,NaN,2,1,2,1,1,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google,2017,10,42,16,0,289,False,False,False,False,False,False,1508112000,2017,10,42,16,0,289,False,False,False,False,False,False,1508169851
4,Organic Search,6960673291025684308,1508190552,1,Chrome,desktop,False,Windows,not available in demo dataset,Americas,Mexico,not available in demo dataset,prod-infinitum.com.mx,not available in demo dataset,Central America,NaN,2,1,2,1,52,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google,2017,10,42,16,0,289,False,False,False,False,False,False,1508112000,2017,10,42,16,0,289,False,False,False,False,False,False,1508190552
5,Referral,0166277907528479249,1508196701,1,Chrome,desktop,False,Macintosh,San Francisco,Americas,United States,San Francisco-Oakland-San Jose CA,unknown.unknown,California,Northern America,NaN,2,1,2,2,12,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(none),/offer/2145,(direct),2017,10,42,16,0,289,False,False,False,False,False,False,1508112000,2017,10,42,16,0,289,False,False,False,False,False,False,1508196701
6,Referral,8349655975937271469,1508152478,1,Chrome,desktop,False,Macintosh,Lond

In [11]:
df_raw['totals.totalTransactionRevenue'].fillna(0, inplace=True)
df_raw['totals.transactions'].fillna(0, inplace=True)
df_raw['totals.transactions'] = df_raw['totals.transactions'].astype('int8')

In [12]:
df_test["visitStartTime"] = pd.to_datetime(df_test["visitStartTime"], infer_datetime_format=True, unit="s")
df_test["date"] = pd.to_datetime(df_test["date"], infer_datetime_format=True, format="%Y%m%d")
add_datepart(df_test, 'date')
add_datepart(df_test, 'visitStartTime')

,channelGrouping,fullVisitorId,visitId,visitNumber,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,visitStartTimeYear,visitStartTimeMonth,visitStartTimeWeek,visitStartTimeDay,visitStartTimeDayofweek,visitStartTimeDayofyear,visitStartTimeIs_month_end,visitStartTimeIs_month_start,visitStartTimeIs_quarter_end,visitStartTimeIs_quarter_start,visitStartTimeIs_year_end,visitStartTimeIs_year_start,visitStartTimeElapsed
0,Organic Search,7460955084541987166,1526099341,2,Chrome,mobile,True,Android,(not set),Asia,India,(not set),unknown.unknown,Delhi,Southern Asia,NaN,4,NaN,3,1,973,NaN,NaN,NaN,(not set),NaN,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,(not set),google,2018,5,19,11,4,131,False,False,False,False,False,False,1525996800,2018,5,19,12,5,132,False,False,False,False,False,False,1526099341
1,Direct,460252456180441002,1526064483,-90,Chrome,desktop,False,Macintosh,San Francisco,Americas,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,NaN,4,NaN,3,1,49,NaN,NaN,NaN,(not set),NaN,NaN,NaN,NaN,NaN,(not set),True,(not set),(none),(not set),(direct),2018,5,19,11,4,131,False,False,False,False,False,False,1525996800,2018,5,19,11,4,131,False,False,False,False,False,False,1526064483
2,Organic Search,3461808543879602873,1526067157,2,Chrome,desktop,False,Chrome OS,not available in demo dataset,Americas,United States,not available in demo dataset,onlinecomputerworks.com,not available in demo dataset,Northern America,NaN,4,NaN,3,1,24,NaN,NaN,NaN,(not set),NaN,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,(not set),google,2018,5,19,11,4,131,False,False,False,False,False,False,1525996800,2018,5,19,11,4,131,False,False,False,False,False,False,1526067157
3,Direct,975129477712150630,1526107551,4,Chrome,mobile,True,iOS,Houston,Americas,United States,Houston TX,(not set),Texas,Northern America,NaN,5,NaN,4,1,25,NaN,NaN,NaN,(not set),NaN,NaN,NaN,NaN,NaN,(not set),True,(not set),(none),(not set),(direct),2018,5,19,11,4,131,False,False,False,False,False,False,1525996800,2018,5,19,12,5,132,False,False,False,False,False,False,1526107551
4,Organic Search,8381672768065729990,1526060254,1,Internet Explorer,tablet,True,Windows,Irvine,Americas,United States,Los Angeles CA,com,California,Northern America,NaN,5,1,4,1,49,NaN,NaN,NaN,(not set),NaN,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,(not set),google,2018,5,19,11,4,131,False,False,False,False,False,False,1525996800,2018,5,19,11,4,131,False,False,False,False,False,False,1526060254
5,Organic Search,2866297766347322467,1526061951,2,Chrome,desktop,False,Windows,not available in demo dataset,Americas,Brazil,not available in demo dataset,vivozap.com.br,not available in demo dataset,South America,NaN,5,NaN,4,1,120,NaN,NaN,NaN,(not set),NaN,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,(not set),google,2018,5,19,11,4,131,False,False,False,False,False,False,1525996800,2018,5,19,11,4,131,False,False,False,False,False,False,1526061951
6,Direct,2235365487897339889,1526062356,1,Chrome,mobile,True,Android,San Jose,Americas,United States,San Francisco-Oakland-San Jose CA,tmodns.net,California,Northern America,NaN,5,1,4,1,58,NaN,Na

In [13]:
from pandas.api.types import is_string_dtype, is_numeric_dtype
def train_cats(df):
    """Change any columns of strings in a panda's dataframe to a column of
    categorical values. This applies the changes inplace.
    """
    for n,c in df.items():
        if is_string_dtype(c): df[n] = c.astype('category').cat.as_ordered()

In [14]:
train_cats(df_raw)
train_cats(df_test)

In [15]:
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import StandardScaler
import sklearn

def numericalize(df, col, name, max_n_cat):
    """ Changes the column col from a categorical type to it's integer codes.
    Parameters:
    -----------
    df: A pandas dataframe. df[name] will be filled with the integer codes from
        col.
    col: The column you wish to change into the categories.
    name: The column name you wish to insert into df. This column will hold the
        integer codes.
    max_n_cat: If col has more categories than max_n_cat it will not change the
        it to its integer codes. If max_n_cat is None, then col will always be
        converted.
    """
    if not is_numeric_dtype(col) and ( max_n_cat is None or len(col.cat.categories)>max_n_cat):
        df[name] = pd.Categorical(col).codes+1

def fix_missing(df, col, name, na_dict):
    """ Fill missing data in a column of df with the median, and add a {name}_na column
    which specifies if the data was missing.
    Parameters:
    -----------
    df: The data frame that will be changed.
    col: The column of data to fix by filling in missing data.
    name: The name of the new filled column in df.
    na_dict: A dictionary of values to create na's of and the value to insert. If
        name is not a key of na_dict the median will fill any missing data. Also
        if name is not a key of na_dict and there is no missing data in col, then
        no {name}_na column is not created.
    """
    if is_numeric_dtype(col):
        if pd.isnull(col).sum() or (name in na_dict):
            df[name+'_na'] = pd.isnull(col)
            filler = na_dict[name] if name in na_dict else col.median()
            df[name] = col.fillna(filler)
            na_dict[name] = filler
    return na_dict

def get_sample(df,n):
    """ Gets a random sample of n rows from df, without replacement.
    Parameters:
    -----------
    df: A pandas data frame, that you wish to sample from.
    n: The number of rows you wish to sample.
    Returns:
    --------
    return value: A random sample of n rows of df.
    """
    idxs = sorted(np.random.permutation(len(df))[:n])
    return df.iloc[idxs].copy()

def scale_vars(df, mapper):
    warnings.filterwarnings('ignore', category=sklearn.exceptions.DataConversionWarning)
    if mapper is None:
        map_f = [([n],StandardScaler()) for n in df.columns if is_numeric_dtype(df[n])]
        mapper = DataFrameMapper(map_f).fit(df)
    df[mapper.transformed_names_] = mapper.transform(df)
    return mapper

def proc_df(df, y_fld=None, skip_flds=None, ignore_flds=None, do_scale=False, na_dict=None,
            preproc_fn=None, max_n_cat=None, subset=None, mapper=None):
    """ proc_df takes a data frame df and splits off the response variable, and
    changes the df into an entirely numeric dataframe. For each column of df 
    which is not in skip_flds nor in ignore_flds, na values are replaced by the
    median value of the column.
    Parameters:
    -----------
    df: The data frame you wish to process.
    y_fld: The name of the response variable
    skip_flds: A list of fields that dropped from df.
    ignore_flds: A list of fields that are ignored during processing.
    do_scale: Standardizes each column in df. Takes Boolean Values(True,False)
    na_dict: a dictionary of na columns to add. Na columns are also added if there
        are any missing values.
    preproc_fn: A function that gets applied to df.
    max_n_cat: The maximum number of categories to break into dummy values, instead
        of integer codes.
    subset: Takes a random subset of size subset from df.
    mapper: If do_scale is set as True, the mapper variable
        calculates the values used for scaling of variables during training time (mean and standard deviation).
    Returns:
    [x, y, nas, mapper(optional)]
    """
    if not ignore_flds: ignore_flds=[]
    if not skip_flds: skip_flds=[]
    if subset: df = get_sample(df,subset)
    else: df = df.copy()
    ignored_flds = df.loc[:, ignore_flds]
    df.drop(ignore_flds, axis=1, inplace=True)
    if preproc_fn: preproc_fn(df)
    if y_fld is None: y = None
    else:
        if not is_numeric_dtype(df[y_fld]): df[y_fld] = pd.Categorical(df[y_fld]).codes
        y = df[y_fld].values
        skip_flds += [y_fld]
    df.drop(skip_flds, axis=1, inplace=True)

    if na_dict is None: na_dict = {}
    else: na_dict = na_dict.copy()
    na_dict_initial = na_dict.copy()
    for n,c in df.items(): na_dict = fix_missing(df, c, n, na_dict)
    if len(na_dict_initial.keys()) > 0:
        df.drop([a + '_na' for a in list(set(na_dict.keys()) - set(na_dict_initial.keys()))], axis=1, inplace=True)
    if do_scale: mapper = scale_vars(df, mapper)
    for n,c in df.items(): numericalize(df, c, n, max_n_cat)
    df = pd.get_dummies(df, dummy_na=True)
    df = pd.concat([ignored_flds, df], axis=1)
    res = [df, y, na_dict]
    if do_scale: res = res + [mapper]
    return res

In [16]:
x_trn, y_trn, nas = proc_df(df_raw, 'totals.transactionRevenue')

In [17]:
from sklearn.ensemble import RandomForestRegressor

In [18]:
X_test, y_test, _ = proc_df(df_test, 'totals.transactionRevenue', nas)

In [19]:
m = RandomForestRegressor(n_estimators=160, max_features=0.5, n_jobs=-1, oob_score=True)

In [20]:
m.fit(x_trn, y_trn)
m.score(x_trn, y_trn)

0.9994974118603477

In [21]:
sub = pd.read_csv('sample_submission_v2.csv', index_col="fullVisitorId")
sub.head()

C:\Users\michael\Anaconda3\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,PredictedLogRevenue
fullVisitorId,
0000018966949534117,0.0
0000039738481224681,0.0
0000073585230191399,0.0
0000087588448856385,0.0
0000149787903119437,0.0


In [22]:
preds = m.predict(X_test)

df_actual_test = df_test.copy()

actual_predicted_revenue = preds
df_actual_test["predicted"] = actual_predicted_revenue

df_actual_test = df_actual_test[["fullVisitorId" , "predicted"]]
df_actual_test["fullVisitorId"] = df_actual_test.fullVisitorId.astype('str')
df_actual_test["predicted"] = df_actual_test.predicted.astype(np.float)
df_actual_test.index = df_actual_test.fullVisitorId
df_actual_test = df_actual_test.drop("fullVisitorId",axis=1)

In [25]:
final_df = df_actual_test.loc[sub.index,:]
final_df = final_df[~final_df.index.duplicated(keep='first')]
final_df = final_df.rename(index=str, columns={"predicted": "PredictedLogRevenue"})
final_df = final_df.fillna(0)
final_df.to_csv('sub.csv')

C:\Users\michael\Anaconda3\envs\py36\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [31]:
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(x_trn, y_trn, test_size=0.25)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_val)

In [33]:
def xgb_evaluate(max_depth, gamma, colsample_bytree):
    params = {'eval_metric': 'rmse',
              'max_depth': int(max_depth),
              'subsample': 0.8,
              'eta': 0.1,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree}
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=100, nfold=3)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

In [34]:
xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 7), 
                                             'gamma': (0, 1),
                                             'colsample_bytree': (0.3, 0.9)})
# Use the expected improvement acquisition function to handle negative numbers
# Optimally needs quite a few more initiation points and number of iterations
xgb_bo.maximize(init_points=3, n_iter=5, acq='ei')

|   iter    |  target   | colsam... |   gamma   | max_depth |
-------------------------------------------------------------
|  1        | -0.1165   |  0.4705   |  0.1845   |  3.195    |
|  2        | -0.1109   |  0.7107   |  0.8492   |  4.476    |
|  3        | -0.1143   |  0.5507   |  0.7306   |  4.299    |
|  4        | -0.1093   |  0.9      |  1.0      |  7.0      |
|  5        | -0.1093   |  0.896    |  0.9945   |  6.977    |
|  6        | -0.1319   |  0.3088   |  0.9865   |  6.98     |
|  7        | -0.1107   |  0.9      |  0.0      |  3.0      |
|  8        | -0.1107   |  0.8949   |  0.9882   |  3.009    |


In [35]:
params = xgb_bo.max['params']
params['max_depth'] = int(params['max_depth'])

In [36]:
model2 = xgb.train(params, dtrain, num_boost_round=250)

# Predict on testing and training set
y_pred = model2.predict(dtest)
y_train_pred = model2.predict(dtrain)

ValueError: Found input variables with inconsistent numbers of samples: [401589, 125000]

In [38]:
# Report testing and training RMSE
print('Validation set: ', np.sqrt(mean_squared_error(y_val, y_pred)))
print('training set: ', np.sqrt(mean_squared_error(y_train, y_train_pred)))

Validation set:  0.10460401243740793
training set:  0.07714914816967178


In [39]:
dtest_final = xgb.DMatrix(X_test)

In [40]:
preds = m.predict(X_test)

df_actual_test = df_test.copy()

actual_predicted_revenue = preds
df_actual_test["predicted"] = actual_predicted_revenue

df_actual_test = df_actual_test[["fullVisitorId" , "predicted"]]
df_actual_test["fullVisitorId"] = df_actual_test.fullVisitorId.astype('str')
df_actual_test["predicted"] = df_actual_test.predicted.astype(np.float)
df_actual_test.index = df_actual_test.fullVisitorId
df_actual_test = df_actual_test.drop("fullVisitorId",axis=1)

In [41]:
final_df = df_actual_test.loc[sub.index,:]
final_df = final_df[~final_df.index.duplicated(keep='first')]
final_df = final_df.rename(index=str, columns={"predicted": "PredictedLogRevenue"})
final_df = final_df.fillna(0)
final_df.to_csv('sub.csv')

C:\Users\michael\Anaconda3\envs\py36\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
